In [1]:
pip install -U gym

Note: you may need to restart the kernel to use updated packages.
  Created wheel for gym: filename=gym-0.18.0-py3-none-any.whl size=1656455 sha256=cfd7c5ad1fb93cb86f6bd65acf591f02746f7ae5d722b114ea95e1bbe625404d
  Stored in directory: c:\users\user\appdata\local\pip\cache\wheels\d8\e7\68\a3f0f1b5831c9321d7523f6fd4e0d3f83f2705a1cbd5daaa79
Successfully built gym
  Attempting uninstall: Pillow
    Found existing installation: Pillow 8.0.1
    Uninstalling Pillow-8.0.1:
      Successfully uninstalled Pillow-8.0.1


In [1]:
import gym
env=gym.make("CartPole-v1")
obs=env.reset()


In [2]:
img=env.render(mode='rgb_array')
img.shape

(400, 600, 3)

In [3]:
env.action_space

Discrete(2)

In [4]:
action=1
obs,reward,done,info=env.step(action)
obs

array([ 0.01486352,  0.21189273, -0.02739943, -0.28239417])

In [5]:
reward

1.0

In [6]:
done

False

In [7]:
info

{}

In [8]:
def basicpo(obs):
    angle=obs[2]
    return 0 if angle<0 else 1
total=[]
for epi in range(500):
    epireward=0
    obs=env.reset()
    for step in range(200):
        action=basicpo(obs)
        obs,reward,done,info=env.step(action)
        epireward+=reward
        if done:
            break
    total.append(epireward)

In [9]:
import numpy as np
np.mean(total),np.std(total),np.min(total),np.max(total)

(41.518, 8.551121329977725, 24.0, 72.0)

In [10]:
env.close()

In [11]:
obs

array([ 0.16108717,  0.61481214, -0.21638788, -0.76284501])

In [12]:
import tensorflow as tf
from tensorflow import keras

ninput=4

model=keras.models.Sequential([
    keras.layers.Dense(5,activation='elu',input_shape=[ninput]),
    keras.layers.Dense(1,activation='sigmoid')
])

In [13]:
def play_one_step(env, obs, model, loss_fn):
    with tf.GradientTape() as tape:
        left_proba = model(obs[np.newaxis])
        action = (tf.random.uniform([1, 1]) > left_proba)
        y_target = tf.constant([[1.]]) - tf.cast(action, tf.float32)
        loss = tf.reduce_mean(loss_fn(y_target, left_proba))
    grads = tape.gradient(loss, model.trainable_variables)
    obs, reward, done, info = env.step(int(action[0, 0].numpy()))
    return obs, reward, done, grads

In [14]:
def play_multiple_episodes(env, n_episodes, n_max_steps, model, loss_fn):
    all_rewards = []
    all_grads = []
    for episode in range(n_episodes):
        current_rewards = []
        current_grads = []
        obs = env.reset()
        for step in range(n_max_steps):
            obs, reward, done, grads = play_one_step(env, obs, model, loss_fn)
            current_rewards.append(reward)
            current_grads.append(grads)
            if done:
                break
        all_rewards.append(current_rewards)
        all_grads.append(current_grads)
    return all_rewards, all_grads

In [15]:
def discount_reward(reward,discount_factor):
    discount=np.array(reward)
    for step in range(len(reward)-2,-1,-1):
        discount[step]+=discount[step+1]*discount_factor
    return discount

def discount_normalize(all_rewards,discount_factor):
    all_discount_rewards=[discount_reward(rewards,discount_factor)for rewards in all_rewards]
    flat_rewards=np.concatenate(all_discount_rewards)
    reward_mean=flat_rewards.mean()
    reward_std=flat_rewards.std()
    return [(discount_reward-reward_mean)/reward_std for discount_reward in all_discount_rewards]

In [16]:
discount_reward([10,0,-50],discount_factor=0.8)

array([-22, -40, -50])

In [17]:
discount_normalize([[10,0,-50],[10,20]],discount_factor=0.8)

[array([-0.28435071, -0.86597718, -1.18910299]),
 array([1.26665318, 1.0727777 ])]

In [18]:
n_iterations = 150
n_episodes_per_update = 10
n_max_steps = 200
discount_rate = 0.95

In [19]:
optimizer=keras.optimizers.Adam(lr=0.01)
loss_fn=keras.losses.binary_crossentropy


In [20]:
env=gym.make('CartPole-v1')
for iterr in  range(n_iter):
    all_rewards,all_grads=play_multiple_episodes(env,n_epiperupdate,n_max_steps,model,loss_fn)
    all_final_rewards=discount_normalize(all_rewards,discount_factor)
    
    all_mean_grads=[]
    for var_index  in range(len(model.trainable_variables)):
        mean_grads=tf.reduce_mean(
            [final_reward * all_grads[epiindex][step][var_index] 
             for epiindex,final_reward in enumerate(all_final_rewards)
                 for step,final_reward in enumerate(final_reward)],axis=0)
        all_mean_grads.append(mean_grads)
    optimizer.apply_gradients(zip(all_mean_grads,model.trainable_variables))
env.close()

NameError: name 'n_iter' is not defined

In [21]:
transition_probabilities = [ # shape=[s, a, s']
        [[0.7, 0.3, 0.0], [1.0, 0.0, 0.0], [0.8, 0.2, 0.0]],
        [[0.0, 1.0, 0.0], None, [0.0, 0.0, 1.0]],
        [None, [0.8, 0.1, 0.1], None]]
rewards = [ # shape=[s, a, s']
        [[+10, 0, 0], [0, 0, 0], [0, 0, 0]],
        [[0, 0, 0], [0, 0, 0], [0, 0, -50]],
        [[0, 0, 0], [+40, 0, 0], [0, 0, 0]]]
possible_actions = [[0, 1, 2], [0, 2], [1]]

In [22]:
qvalue=np.full((3,3),-np.inf)
for state,actions in enumerate (possible_actions):
    qvalue[state,actions]=0.0

In [26]:
gamma=0.95

for iter in range(50):
    qprev=qvalue.copy()
    for s in range(3):
        for a in possible_actions[s]:
            qvalue[s,a]=np.sum([
                transition_probabilities[s][a][sp]
                * (rewards[s][a][sp]+gamma*np.max(qprev[sp]))
                for sp in range(3)
            ])
        

In [27]:
qvalue

array([[21.77572778, 20.68076527, 16.74407361],
       [ 0.99730695,        -inf,  1.05629796],
       [       -inf, 53.74997271,        -inf]])

In [28]:
np.argmax(qvalue,axis=1)

array([0, 2, 1], dtype=int64)

In [29]:
def step (state,action):
    probas=transition_probabilities[state][action]
    nextstate=np.random.choice([0,1,2],p=probas)
    reward=rewards[state][action][nextstate]
    return nextstate,reward

In [30]:
def explorpolicy(state):
    return np.random.choice(possible_actions[state])

In [31]:
alpha0=0.05
decay=0.005
gamma=0.9
state=0

for iter in range(10000):
    action=explorpolicy(state)
    nextstate,reward=step(state,action)
    nextvalue=np.max(qvalue[nextstate])
    alpha=alpha0/(1+iter*decay)
    qvalue[state,action]*=1-alpha
    qvalue[state,action]+=alpha*(reward+gamma*nextvalue)
    state=nextstate

In [32]:
env=gym.make('CartPole-v0')
input_shape=[4]
n_outputs=2

model=keras.models.Sequential([
    keras.layers.Dense(32,activation='elu',input_shape=input_shape),
    keras.layers.Dense(32,activation='elu'),
    keras.layers.Dense(n_outputs)
])

In [33]:
def epsilonpolicy(state,epsilon=0):
    if np.random.rand()<epsilon:
        return np.random.randint(2)
    else:
        q_value=model.predict(state[np.newaxis])
        return np.argmax(q_value[0])
        

In [34]:
from collections import deque

replaybuffer=deque(maxlen=2000)

In [35]:
def sample_exper(batch_size):
    indices=np.random.randint(len(replaybuffer),size=batch_size)
    batch=[replaybuffer[index] for index in indices]
    states,actions,rewards,next_states,dones=[
        np.array([exper[fieldindex]for exper in batch])
        for fieldindex in range(5)]
    return states,actions,rewards,next_states,dones

In [40]:
def playonestep(env,state,epsilon):
    action=epsilonpolicy(state,epsilon)
    nextstate,reward,done,info=env.step(action)
    replaybuffer.append((state,action,reward,nextstate,done))
    return nextstate,reward,done,info

In [43]:
batchsize=32
discountfactor=0.95
optimizer=keras.optimizers.Adam(lr=1e-3)
loss_fn=keras.losses.mean_squared_error

def training_step(batchsize):
    exper=sample_exper(batchsize)
    states,actions,rewards,next_states,dones=exper
    nextqvalue=model.predict(next_states)
    maxnextqvalue=np.max(nextqvalue,axis=1)
    targetqvalues=(rewards+(1-dones)*discountfactor*maxnextqvalue )
    targetqvalues=targetqvalues.reshape(-1,1)
    mask=tf.one_hot(actions,n_outputs)
    with tf.GradientTape() as tape:
        allqvalue=model(states)
        qvalue=tf.reduce_sum(allqvalue*mask,axis=1,keepdims=True)
        loss=tf.reduce_mean(loss_fn(targetqvalues,qvalue))
    grads=tape.gradient(loss,model.trainable_variables)
    optimizer.apply_gradients(zip(grads,model.trainable_variables))

In [44]:
for episode in range(600):
    obs=env.reset()
    for step in range(200):
        epsilon=max(1-episode/500,0.01)
        obs,reward,done,info=playonestep(env,obs,epsilon)
        if done:
            break
    if episode>50:
        training_step(batchsize)

In [51]:
from tf_agents.environments import suite_gym
env=suite_gym.load('Breakout-v4')
env

FileNotFoundError: Could not find module 'C:\Users\User\anaconda3\lib\site-packages\atari_py\ale_interface\ale_c.dll' (or one of its dependencies). Try using the full path with constructor syntax.
  In call to configurable 'load' (<function load at 0x000001C74FEB1820>)